In [1]:
conda update -n base -c defaults conda












pyopenssl-23.2.0     | 96 KB     | ######1                               |  17% 










conda-23.7.2         | 1.0 MB    | ####                                  |  11% 
















certifi-2023.7.22    | 153 KB    | ##################################### | 100% 













tzdata-2023c         | 116 KB    | ##################################### | 100% 
















certifi-2023.7.22    | 153 KB    | ##################################### | 100% 













tzdata-2023c         | 116 KB    | ##################################### | 100% 















jsonpatch-1.32       | 15 KB     | ##################################### | 100% 

















pyopenssl-23.2.0     | 96 KB     | ##################################### | 100% 



python-3.9.17        | 25.0 MB   | #5                                    |   4% 



















xz-5.4.2             | 642 KB    | 9                                     |   2% 


















openssl-3.0.10       | 5.2 MB    | 1   

In [2]:
import paddle, imghdr, os, json, time
from paddle.io import Dataset
from paddle.vision import transforms as T
from PIL import Image
from paddle.io import DataLoader
from paddle.vision import models
from paddlenlp.transformers import ErnieMModel, ErnieMTokenizer
import paddle.nn as nn 
from tqdm import tqdm
from paddlenlp.transformers import LinearDecayWithWarmup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [3]:
with open('queries_dataset_merge/dataset_items_train.json') as train_file:
    train_data_items = json.load(train_file)

with open('queries_dataset_merge/dataset_items_val.json') as val_file:
    val_data_items = json.load(val_file)

with open('queries_dataset_merge/dataset_items_test.json') as test_file:
    test_data_items = json.load(test_file)

print(train_data_items.keys())
print(train_data_items['0'])

dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', 

In [4]:
# create dataset

def process_string(input_str):
    input_str = input_str.replace('&#39;', '')
    input_str = input_str.replace('<b>','')
    input_str = input_str.replace('</b>','')
    return input_str


class NewsContextDatasetEmbs(Dataset):
    def __init__(self, context_data_items_dict, queries_root_dir, split):
        super().__init__()
        self.context_data_items_dict = context_data_items_dict
        self.queries_root_dir = queries_root_dir
        self.idx_to_keys = list(context_data_items_dict.keys())
        self.transform = T.Compose([T.Resize(256),
                                    T.CenterCrop(224),
                                    T.ToTensor(),
                                    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                    ])
        self.split = split

    def __len__(self):
        return len(self.context_data_items_dict)

    def load_img_pil(self, image_path):
        if imghdr.what(image_path) == 'gif':
            try:
                img = Image.open(image_path)
                return img.convert('RGB')
            except:
                return None
        img = Image.open(image_path)
        return img.convert('RGB')

    def load_img_direct_search(self, item_folder_path, direct_anno_dict):
        keys_to_check = ['images_with_captions', 'images_with_no_captions', 'images_with_caption_matched_tags']
        count = 0
        list_imgs_tensor = []
        for key1 in keys_to_check:
            if key1 in list(direct_anno_dict.keys()):
                for page in direct_anno_dict[key1]:
                    image_path = os.path.join(item_folder_path, page['image_path'].split('/')[-1])
                    try:
                        pil_img = self.load_img_pil(image_path)
                    except Exception as e:
                        print(e)
                        print(image_path)
                    if pil_img is not None:
                        trans_img = self.transform(pil_img)
                        count += 1
                        list_imgs_tensor.append(trans_img)
                    else: continue
        stacked_tensor = paddle.stack(list_imgs_tensor, axis=0)
        return stacked_tensor

    def load_caption_direct_search(self, direct_anno_dict):
        keys_to_check = ['images_with_captions', 'images_with_no_captions', 'images_with_caption_matched_tags']
        captions = [] # caption = ['']
        for key1 in keys_to_check:
            if key1 in list(direct_anno_dict.keys()):
                for page in direct_anno_dict[key1]:
                    if 'page_title' in list(page.keys()):
                        captions.append(process_string(page['page_title']))
                    if 'caption' in list(page.keys()):
                        sub_captions_list = []
                        processed_captions = []
                        for key2 in page['caption'].keys():
                            sub_caption = page['caption'][key2]
                            if sub_caption not in processed_captions:
                                processed_captions.append(sub_caption)
                                sub_captions_list.append(process_string(sub_caption))
                            else: continue
                        captions = captions + sub_captions_list
        return captions


    def load_caption_inv_search(self, inv_anno_dict):
        pages_with_caption_keys = ['all_fully_matched_captions', 'all_partially_matched_captions']
        captions = [] # caption = ['']
        for key1 in pages_with_caption_keys:
            if key1 in list(inv_anno_dict.keys()):
                for page in inv_anno_dict[key1]:
                    if 'title' in list(page.keys()):
                        item = page['title']
                        item = process_string(item)
                        captions.append(item)

                    if 'caption' in list(page.keys()):
                        sub_captions_list = []
                        processed_captions = []
                        for key2 in page['caption'].keys():
                            sub_caption = page['caption'][key2]
                            if sub_caption not in processed_captions:
                                processed_captions.append(sub_caption)
                                sub_caption = process_string(sub_caption)
                                sub_captions_list.append(sub_caption)
                            else: continue
                        captions = captions + sub_captions_list
        
        pages_with_title_only_keys = ['partially_matched_no_text', 'fully_matched_no_text']
        for key3 in pages_with_title_only_keys:
            if key3 in list(inv_anno_dict.keys()):
                for page in inv_anno_dict[key3]:
                    if 'title' in list(page.keys()):
                        title = process_string(page['title'])
                        captions.append(title)
        return captions

    def load_queries(self, idx):
        caption = self.context_data_items_dict[idx]['caption']
        image_path = os.path.join(self.queries_root_dir, self.context_data_items_dict[idx]['image_path'])
        pil_img = self.load_img_pil(image_path)
        trans_img = self.transform(pil_img)
        return trans_img, caption

    def __getitem__(self, idx):
        key = self.idx_to_keys[idx] #数据样本量
        item = self.context_data_items_dict.get(str(key))
        if self.split == 'train' or self.split == 'val':
            label = paddle.to_tensor(int(item['label']))
            direct_path = os.path.join(self.queries_root_dir, item['direct_path'])
            inverse_path = os.path.join(self.queries_root_dir, item['inv_path'])
            direct_anno_dict = json.load(open(os.path.join(direct_path, 'direct_annotation.json')))
            inverse_anno_dict = json.load(open(os.path.join(inverse_path, 'inverse_annotation.json')))
            captions = self.load_caption_inv_search(inverse_anno_dict)
            captions += self.load_caption_direct_search(direct_anno_dict)
            imgs = self.load_img_direct_search(direct_path, direct_anno_dict) # imgs是依据声明搜索到的图片
            qImg, qCap = self.load_queries(key) # qimg和qcap是img文件夹下的图片和对应的申明
            samples = {'label': label, 'caption': captions,'imgs': imgs,  'qImg': qImg, 'qCap': qCap}
        else:
            direct_path = os.path.join(self.queries_root_dir, item['direct_path'])
            inverse_path = os.path.join(self.queries_root_dir, item['inv_path'])
            direct_anno_dict = json.load(open(os.path.join(direct_path, 'direct_annotation.json')))
            inverse_anno_dict = json.load(open(os.path.join(inverse_path, 'inverse_annotation.json')))
            captions = self.load_caption_inv_search(inverse_anno_dict)
            captions += self.load_caption_direct_search(direct_anno_dict)
            imgs = self.load_img_direct_search(direct_path, direct_anno_dict) # imgs是依据声明搜索到的图片
            qImg, qCap = self.load_queries(key)  # qimg和qcap是img文件夹下的图片和对应的声明
            samples = {'caption': captions,'imgs': imgs,  'qImg': qImg, 'qCap': qCap}
        return samples, len(captions), imgs.shape[0]

In [5]:
# create dataset we need
train_dataset = NewsContextDatasetEmbs(train_data_items, 'queries_dataset_merge', 'train')
val_dataset = NewsContextDatasetEmbs(val_data_items, 'queries_dataset_merge', 'val')
test_dataset = NewsContextDatasetEmbs(test_data_items, 'queries_dataset_merge', 'test')

In [6]:
# test the dataset
for step, data_batch in enumerate(test_dataset, start=1):
    print(step, data_batch[2])
    break

for step, data_batch in enumerate(train_dataset, start=1):
    print(step, data_batch[0].keys())
    break

W0821 17:37:57.410531   166 gpu_resources.cc:119] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.2, Runtime API Version: 11.6
W0821 17:37:57.439164   166 gpu_resources.cc:149] device: 0, cuDNN Version: 8.4.


1 3
1 dict_keys(['label', 'caption', 'imgs', 'qImg', 'qCap'])


In [7]:
# create dataloader
def collate_context_bert_train(data_batch):
    samples = [ret[0] for ret in data_batch]
    max_captions_len = max([ret[1] for ret in data_batch])
    max_imgs_len = max([ret[2] for ret in data_batch]) # 得到最多的imgs搜索数量
    qImg_batch = []
    qCap_batch = []
    imgs_batch = []
    caps_batch = []
    labels_batch = []

    for sample in samples:
        labels_batch.append(sample['label'])
        captions = sample['caption']
        cap_len = len(captions)
        for i in range(max_captions_len-cap_len):
            captions.append('')
            
        pad_size = (max_imgs_len-sample['imgs'].shape[0], sample['imgs'].shape[1], sample['imgs'].shape[2], sample['imgs'].shape[3])
        pad_imgs = paddle.concat((sample['imgs'], paddle.zeros(pad_size)), axis=0)

        imgs_batch.append(pad_imgs)
        caps_batch.append(captions)
        qImg_batch.append(sample['qImg'])
        qCap_batch.append(sample['qCap'])
        pass

    imgs_batch = paddle.stack(imgs_batch, axis=0)
    qImg_batch = paddle.stack(qImg_batch, axis=0)
    labels_batch = paddle.stack(labels_batch, axis=0)

    return labels_batch, caps_batch, imgs_batch, qCap_batch, qImg_batch

def collate_context_bert_test(data_batch):
    samples = [ret[0] for ret in data_batch]
    max_captions_len = max([ret[1] for ret in data_batch])
    max_images_len = max([ret[2] for ret in data_batch])
    qCap_batch = []
    qImg_batch = []
    img_batch = []
    cap_batch = []

    for j in range(0,len(samples)):  
        sample = samples[j]    
        captions = sample['caption']
        cap_len = len(captions)
        for i in range(0,max_captions_len-cap_len):
            captions.append("")
        if len(sample['imgs'].shape) > 2:
            padding_size = (max_images_len-sample['imgs'].shape[0],sample['imgs'].shape[1],sample['imgs'].shape[2],sample['imgs'].shape[3])
        else:
            padding_size = (max_images_len-sample['imgs'].shape[0],sample['imgs'].shape[1])
        padded_mem_img = paddle.concat((sample['imgs'], paddle.zeros(padding_size)),axis=0)
        img_batch.append(padded_mem_img)
        cap_batch.append(captions)
        qImg_batch.append(sample['qImg'])
        qCap_batch.append(sample['qCap'])        
    img_batch = paddle.stack(img_batch, axis=0)
    qImg_batch = paddle.stack(qImg_batch, axis=0)
    return cap_batch, img_batch, qCap_batch, qImg_batch

In [8]:
# create dataloader
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True, collate_fn=collate_context_bert_train, return_list=True)
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=True, collate_fn=collate_context_bert_train, return_list=True)
test_dataloader = DataLoader(test_dataset, batch_size=2, shuffle=False, collate_fn = collate_context_bert_test, return_list=True)

In [9]:
# test dataloader
for step, batch in enumerate(test_dataloader, start=1):
    print(step, batch[0])
    break

1 [['Boston Orange  波士頓菊子: 朱学渊  - 為中國史學的實證化而努力', '新华每日电讯-微报纸-2022年01月28日', '新华每日电讯-微报纸-2021年11月19日', '', '', '', ''], ['Reddit - Dive into anything', '独立站如何做竞争对手调查？ - 知无不言跨境电商社区', '图八.png', '没想到阎维文女婿这么厉害：罕见曝光女婿身份！怪不得这么着急嫁-综艺-完整版视频在线观看-爱奇艺', '导师的水平是否决定了一个博士生的天花板？怎么才能雄起呢？ - 知乎', '怪不得可以吸引斯蒂芬妮，这样的HHH简直太强了！_兰迪', '丰田汉兰达，车身线条流畅大气，棱角分明 - 知乎']]


In [10]:
# create model

class Img_Resnet(nn.Layer):
    def __init__(self, resnet_type='resnet101'):
        super().__init__()
        if resnet_type == 'resnet101':
            resnet = models.resnet101(pretrained=True)
        modules_list = list(resnet.children())[:-2] # here ignore the pool and fc in the last 2 layer
        self.resnet = nn.Sequential(*modules_list)
        self.adaptive_pool = nn.AdaptiveAvgPool2D((1,1))

    def forward(self, input_imgs, features='pool'):
        out = self.resnet(input_imgs)
        if features == 'pool':
            out = self.adaptive_pool(out)
            out = paddle.reshape(out, (out.shape[0], out.shape[1]))
        return out

class NetWork(nn.Layer):
    def __init__(self):
        super().__init__()
        self.ernie = ErnieMModel.from_pretrained('ernie-m-base')
        self.tokenizer = ErnieMTokenizer.from_pretrained('ernie-m-base')
        self.resnet = Img_Resnet()
        self.classifier1 = nn.Linear(2*(768+2048),1024)
        self.classifier2 = nn.Linear(1024,3)
        self.attention_text = nn.MultiHeadAttention(768,16)
        self.attention_image = nn.MultiHeadAttention(2048,16)
        self.resnet.eval()

    def forward(self, qCap,qImg,caps,imgs):
        self.resnet.eval()
        encode_dict_qcap = self.tokenizer(text = qCap ,max_length = 128 ,truncation=True, padding='max_length')
        input_ids_qcap = encode_dict_qcap['input_ids']
        input_ids_qcap = paddle.to_tensor(input_ids_qcap)
        qcap_feature, pooled_output= self.ernie(input_ids_qcap)
        
        caps_feature = []
        for i, caption in enumerate(caps):
            encode_dict_cap = self.tokenizer(text = caption ,max_length = 128 ,truncation=True, padding='max_length')
            input_ids_caps = encode_dict_cap['input_ids']
            input_ids_caps = paddle.to_tensor(input_ids_caps)
            cap_feature, pooled_output= self.ernie(input_ids_caps) #(b,length,dim)
            caps_feature.append(cap_feature)
        caps_feature = paddle.stack(caps_feature,axis=0)
        caps_feature = caps_feature.mean(axis=1)

        caps_feature = self.attention_text(qcap_feature,caps_feature,caps_feature)

        imgs_features = []
        for img in imgs:
            imgs_feature = self.resnet(img)
            imgs_features.append(imgs_feature)
        imgs_features = paddle.stack(imgs_features,axis=0)

        qImg_features = []
        for qimage in qImg:
            qImg_feature = self.resnet(qimage.unsqueeze(axis=0))
            qImg_features.append(qImg_feature)
        qImg_feature = paddle.stack(qImg_features,axis=0)

        imgs_features = self.attention_image(qImg_feature, imgs_features, imgs_features) 
        feature = paddle.concat(x=[qcap_feature[:,0,:], caps_feature[:,0,:], qImg_feature.squeeze(1), imgs_features.squeeze(1)], axis=-1)
        logits = self.classifier1(feature)
        logits = self.classifier2(logits)
        return logits

In [ ]:
# create model
model = NetWork()
# print(model)

In [ ]:
# train model config

epochs = 2
num_training_steps = len(train_dataloader) * epochs
warmup_steps = int(num_training_steps*0.1)
lr_scheduler = LinearDecayWithWarmup(1e-6, num_training_steps, warmup_steps)

save_dir ="checkpoint/"
best_dir = "best_model"
os.makedirs(save_dir,exist_ok=True)
os.makedirs(best_dir,exist_ok=True)

decay_params = [
    p.name for n, p in model.named_parameters()
    if not any(nd in n for nd in ["bias", "norm"])
]

optimizer = paddle.optimizer.AdamW(
    learning_rate=lr_scheduler,
    parameters=model.parameters(),
    weight_decay=1.2e-4,
    apply_decay_param_fun=lambda x: x in decay_params)

criterion = paddle.nn.loss.CrossEntropyLoss()
metric = paddle.metric.Accuracy()

In [ ]:
# evaluate func

@paddle.no_grad()
def evaluate(model, criterion, metric, data_loader):
    model.eval()
    metric.reset()
    losses = []
    for batch in data_loader:      
        labels, cap_batch, img_batch, qCap_batch, qImg_batch = batch
        logits = model(qCap=qCap_batch,qImg=qImg_batch,caps=cap_batch,imgs=img_batch)
        loss = criterion(logits, labels)
        losses.append(loss.numpy())
        correct = metric.compute(logits, labels)
        metric.update(correct)
        accu = metric.accumulate()
    print("eval loss: %.5f, accu: %.5f" % (np.mean(losses), accu))
    model.train()
    metric.reset()
    return np.mean(losses), accu

In [ ]:
# train func

def train_epoch(model, criterion, metric, val_dataloader,train_dataloader):
    model.train()
    print("train run start")
    global_step = 0
    tic_train = time.time()
    best_accuracy=0.0

    for epoch in range(1, epochs+1):
        for step, batch in enumerate(train_dataloader, start=1):
            labels_batch, caps_batch, imgs_batch, qCap_batch, qImg_batch = batch
            output = model(qCap=qCap_batch, qImg=qImg_batch, caps=caps_batch, imgs=imgs_batch)
            loss = criterion(output, labels_batch)
            correct = metric.compute(output, labels_batch)
            metric.update(correct)
            acc = metric.accumulate()

            global_step += 1
            if global_step % 100 == 0:
                print(
                    "global step %d, epoch: %d, batch: %d, loss: %.5f, accu: %.5f, speed: %.2f step/s"
                    % (global_step, epoch, step, loss, acc,
                        10 / (time.time() - tic_train)))
                tic_train = time.time()

            loss.backward()
            optimizer.step()
            lr_scheduler.step()
            optimizer.clear_grad()

            
            if global_step % len(train_dataloader) == 0:
                eval_loss,eval_accu=evaluate(model, criterion, metric, val_dataloader)
                save_param_path = os.path.join(save_dir+str(epoch), 'model_state.pdparams')
                paddle.save(model.state_dict(), save_param_path)
                if(best_accuracy<eval_accu):
                    best_accuracy=eval_accu
                    # 保存模型
                    save_param_path = os.path.join(best_dir, 'model_best.pdparams')
                    paddle.save(model.state_dict(), save_param_path)


In [ ]:
# train_epoch(model, criterion, metric, val_dataloader,train_dataloader) 

In [11]:
# predict 
model = NetWork()

params_path = 'best_model/model_best.pdparams'
if params_path and os.path.isfile(params_path):
    weight_dict = paddle.load(params_path)
    model.set_dict(weight_dict)
    print("Loaded parameters from %s" % params_path)



[2023-08-21 17:38:11,379] [    INFO] - Model config ErnieMConfig {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 514,
  "model_type": "ernie_m",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "paddlenlp_version": null,
  "type_vocab_size": 16,
  "vocab_size": 250002
}

[2023-08-21 17:38:11,382] [    INFO] - Configuration saved in /home/aistudio/.paddlenlp/models/ernie-m-base/config.json
[2023-08-21 17:38:11,384] [    INFO] - Downloading ernie_m_base.pdparams from https://paddlenlp.bj.bcebos.com/models/transformers/ernie_m/ernie_m_base.pdparams
100%|██████████| 1.04G/1.04G [01:17<00:00, 14.4MB/s] 
[2023-08-21 17:39:32,421] [ WARNING] - Some weights of the model checkpoint at ernie-m-base were not used when initializing ErnieMModel: ['cls.predictions.layer_norm.weight', 'cls.predictions.decoder_bias', '

Loaded parameters from best_model/model_best.pdparams


In [12]:
import paddle.nn.functional as F

model.eval()
results = []

count = 0

for batch in test_dataloader:
    count += 1
    cap_batch, img_batch, qCap_batch, qImg_batch = batch
    logits = model(qCap=qCap_batch,qImg=qImg_batch,caps=cap_batch,imgs=img_batch)
    probs = F.softmax(logits, axis=-1)
    label = paddle.argmax(probs, axis=1).numpy()
    results += label.tolist()
    print(count)
print(results[:5])
print(len(results))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [13]:
id_list=range(len(results))
print(id_list)
frame = pd.DataFrame({'id':id_list,'label':results})
frame.to_csv("result.csv",index=False,sep=',')

range(0, 1129)
